In [ ]:
import numpy as np
import os
import re
import itertools
import collections
import tqdm
import pickle
import datetime

In [ ]:
arquivos95 = list(os.walk('Bases/folha95/'))[0][2]
arquivos94 = list(os.walk('Bases/folha94/'))[0][2]

In [ ]:
corpus = ''
for a in arquivos95:
        with open('Bases/folha95/' + a,'r', encoding='iso-8859-1') as f:
                corpus = corpus + f.read()
for a in arquivos94:
        with open('Bases/folha94/' + a,'r', encoding='iso-8859-1') as f:
                corpus = corpus + f.read()

In [ ]:
def tokenize (txt):
    p = re.finditer ('(?:<TEXT>).*?(?:</TEXT>)',txt,flags=re.DOTALL|re.MULTILINE)
    for a in p:
        for x in re.finditer('[a-zA-ZâêîôûáéíóúàèìòùãẽĩõũçÁÉÍÓÚÀÈÌÒÙÃẼĨÕŨÂÊÎÔÛÇ]+|<TEXT>|</TEXT>|\.',a.group(),flags=re.DOTALL|re.MULTILINE):
            t = x.group()
            if t=='<TEXT>':
                yield '<s>'
                yield '<s>'
                yield '<s>'             
            elif t == '</TEXT>':
                yield '</s>'
                yield '</s>'
                yield '</s>' 
            elif t == '.':
                yield '</s>'
                yield '</s>'
                yield '</s>'  
                yield '<s>'
                yield '<s>'
                yield '<s>'
            else:
                yield t.lower()

In [ ]:
inverseDict={}
total=0
for p in tokenize(corpus):
    if p not in inverseDict:
        inverseDict[p]=total
        total+=1

In [ ]:
len(inverseDict)
print (inverseDict)

In [ ]:
def slidingWindow(winsize):
    def itt(tokenizedText):
        iters=itertools.tee(tokenizedText,winsize)
        for x in range(winsize):
            for i in range(x):
                next(iters[x])
        while True:
            try:
                yield tuple([next(it) for it in iters])
            except StopIteration:
                return
    return itt

In [ ]:
f4 = slidingWindow(4)
quatro = f4(tokenize(corpus))

In [ ]:
next(quatro)

In [ ]:
f4 = slidingWindow(4)
f3 = slidingWindow(3)
f2 = slidingWindow(2)
quatro = f4(tokenize(corpus))
tres = f3(tokenize(corpus))
dois = f2(tokenize(corpus))
um = tokenize(corpus)

In [ ]:
total = sum(1 for _ in tokenize(corpus))

In [ ]:
print (total)

In [ ]:
contador={}

In [ ]:
for a,b,c,d in tqdm.tqdm(zip(um,dois,tres,quatro),total=total):
    if a not in contador:
        contador[a]={None: 1}
    else:
        contador[a][None]+=1
        
    if b[1] not in contador[b[0]]:
        contador[b[0]][b[1]]={None: 1}
    else:
        contador[b[0]][b[1]][None]+=1
    
    if c[2] not in contador[c[0]][c[1]]:
        contador[c[0]][c[1]][c[2]] = {None: 1}
    else:
        contador[c[0]][c[1]][c[2]][None] += 1
        
    if d[3] not in contador[d[0]][d[1]][d[2]]:
        contador[d[0]][d[1]][d[2]][d[3]] = 1
    else:
        contador[d[0]][d[1]][d[2]][d[3]] += 1

In [ ]:
contador['<s>']['<s>']['<s>']['a']/contador['<s>']['<s>']['<s>'][None]

In [ ]:
with open('Bases/recomend/rec.pickle', 'wb') as handle:
    pickle.dump(contador, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def analisaFrase(x):
    xx = tokenize(x)
    tres = f3(tokenize(x))
    dois = f2(tokenize(x))
    next(dois)
    um = tokenize(x)
    next(um)
    next(um)
    prod=0
    w1,w2,w3=0.1,0.2,0.7
    i=0
    for w in zip(um, dois,tres):
        p3=0
        p2=0
        
        if contador[w[2][0]][w[2][1]][None] > 0:
            p3 = contador[w[2][0]][w[2][1]][w[2][2]][None]/contador[w[2][0]][w[2][1]][None]
            
        if contador[w[1][0]][None] > 0:
            p2 = contador[w[1][0]][w[1][1]][None]/contador[w[1][0]][None]
            
        p1 = contador[w[0]][None]/total
        
        prod = prod + np.log(w1*p1 + w2*p2 + w3*p3)
        i+=1
    return 1/((np.exp(prod))**(1/i))